## Food Inspection: Data Extraction

The purpose of this notebook is to collect the data from the two primary sources and create a single data source.  This product will be updated and improved over time and may eventually become a database to better manage data updates.  Listed below are some initial features that will be explored but for more details about the features, review the contents of the `feature_analysis.ipynb` notebook.  The features included in the dataset will change over time as more data is collected.  

**Initial Features**  
1.  name  
1.  license number  
1.  result (pass/fail)
1.  business age  
1.  number of chains
1.  risk  
1.  ward / neighborhood
1.  license code  
1.  renew  
1.  conditional approved  
1.  business activity


In [2]:
# Import required libraries

# # Code formatter
# # !pip3 install nb_black
# %load_ext nb_black

# eda tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import re

# hide jupyter lab warnings
import warnings
warnings.filterwarnings('ignore')

# expand the number of dataframe columns visible
pd.options.display.max_columns = 100

# make sound when this code executes: Audio(sound_file, autoplay=True)
from IPython.display import Audio
sound_file = './sound/chord.wav'

# display package informatin
# !conda install -c conda-forge session-info
import session_info
session_info.show()


In [4]:
# helper function
def date_select(row):
    """This function returns the the most appropriate date related to the license start date.  The function 
    first checks the license start date column and if that is a valid date then returns it.  The order of 
    importance is `license_start_date` > `license_issued_date` > `app_complete` > date_issued > license_end_date.  
    If none of those columns has a valid date then it returns `NaT`

    """
    if str(row.license_start_date) != 'NaT': 
        temp = row.license_start_date
    elif str(row.license_issued_date) != 'NaT':
        temp = row.license_issued_date
    elif str(row.app_complete) != 'NaT':
        temp = row.app_complete
    elif str(row.date_issued) != 'NaT':
        temp = row.date_issued
    elif str(row.license_end_date) != 'NaT':
        temp = row.license_end_date
    else:
        temp = np.datetime64('nat')
    return temp

## Clean Food Safety Inspection Data

In [5]:
# Read data
inspections_df = pd.read_csv('data/original/food_inspections.csv', parse_dates=['Inspection Date'])
print(f'Original number of records: {len(inspections_df)}')
inspections_df.dropna(subset='License #', inplace=True)
inspections_df['License #'].astype('int32')

restaurant_df = inspections_df[inspections_df['Facility Type'] == 'Restaurant'].copy()
restaurant_df = restaurant_df[~(restaurant_df['License #'] == 0)]
restaurant_df.drop(['DBA Name', 'Location'], axis=1, inplace=True)
restaurant_df = restaurant_df[~restaurant_df['Results'].isin(['Out of Business','No Entry', 'Not Ready', 'Business Not Located'])]
restaurant_df.columns = ['inspect_id', 'aka_name', 'license_num', 'facility_type', 'risk',
                           'address', 'city', 'state', 'zipcode', 'inspect_date', 'inspect_type',
                           'results', 'violations', 'lat', 'lon']

# strip white space from object types
columns = restaurant_df.select_dtypes(['object']).columns
restaurant_df[columns] = restaurant_df[columns].apply(lambda x: x.str.strip())

# restaurant_df.dropna(subset=['violations'], inplace=True)
restaurant_df['violations'].fillna('-99.  No violations reported', inplace=True)
restaurant_df['violations_list'] = restaurant_df['violations'].apply(lambda x: x.split("|"))
restaurant_df['violation_count'] = restaurant_df['violations_list'].apply(lambda x: len(x))

restaurant_df['violation_number'] = restaurant_df['violations_list'].apply(lambda x: [re.findall(r'\b\d+\b',i)[0] for i in x])
restaurant_df.head(3)

restaurant_df['vl_must_comply_list'] = restaurant_df['violations_list'].apply(lambda x: [ 'MUST COMPLY' in i for i in x])
restaurant_df['vl_must_comply_count'] = restaurant_df['vl_must_comply_list'].apply(lambda x: sum(x))

restaurant_df['vl_instructed_comply_list'] = restaurant_df['violations_list'].apply(lambda x: ['INSTRUCTED TO COMPLY' in i for i in x])
restaurant_df['vl_instructed_comply_count'] = restaurant_df['vl_instructed_comply_list'].apply(lambda x: sum(x))

restaurant_df['vl_not_cited_list'] = restaurant_df['violations_list'].apply(lambda x: ['NO CITATION ISSUED' in i for i in x])
restaurant_df['vl_citation_count'] = restaurant_df['vl_not_cited_list'].apply(lambda x: len(x) - sum(x))  

restaurant_df['year'] = restaurant_df['inspect_date'].dt.strftime('%Y').astype('int')
restaurant_df['month'] = restaurant_df['inspect_date'].dt.strftime('%m').astype('int')

restaurant_df.drop_duplicates(subset=['aka_name','license_num','facility_type','risk','address','city','state',
                                     'zipcode','inspect_date','results','lat','lon'], 
                              keep='last',
                              inplace=True)

print(f'Number of records after cleaning:  {len(restaurant_df)}')

restaurant_df.head(2)

Original number of records: 258053
Number of records after cleaning:  150015


,inspect_id,aka_name,license_num,facility_type,risk,address,city,state,zipcode,inspect_date,inspect_type,results,violations,lat,lon,violations_list,violation_count,violation_number,vl_must_comply_list,vl_must_comply_count,vl_instructed_comply_list,vl_instructed_comply_count,vl_not_cited_list,vl_citation_count,year,month
5,2579896,LAS ISLAS MARIAS,1943409.0,Restaurant,Risk 1 (High),2400 S PULASKI RD,CHICAGO,IL,60623.0,2023-08-10,Canvass Re-Inspection,Fail,-99. No violations reported,41.847858,-87.724795,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8
8,2579839,QUERETACO,2918599.0,Restaurant,Risk 2 (Medium),2247 E 71ST ST,CHICAGO,IL,60649.0,2023-08-09,License,Pass,-99. No violations reported,41.766032,-87.569550,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8


In [6]:
restaurant_df.dtypes

inspect_id                             int64
aka_name                              object
license_num                          float64
facility_type                         object
risk                                  object
address                               object
city                                  object
state                                 object
zipcode                              float64
inspect_date                  datetime64[ns]
inspect_type                          object
results                               object
violations                            object
lat                                  float64
lon                                  float64
violations_list                       object
violation_count                        int64
violation_number                      object
vl_must_comply_list                   object
vl_must_comply_count                   int64
vl_instructed_comply_list             object
vl_instructed_comply_count             int64
vl_not_cit

In [7]:
restaurant_df.shape

(150015, 26)

## Clean Business License Data

In [8]:
# Read data
business_info = pd.read_csv('data/original/all_licensed_businesses.csv', parse_dates=['PAYMENT DATE','APPLICATION REQUIREMENTS COMPLETE','APPLICATION CREATED DATE','LICENSE TERM START DATE','LICENSE TERM EXPIRATION DATE','LICENSE APPROVED FOR ISSUANCE','DATE ISSUED'])
# Note:  APPLICATION CREATED DATE has NaT values as warned below

print(f'Original number of records: {len(business_info)}')

business_info.drop(['ID', 'LICENSE STATUS', 'ACCOUNT NUMBER', ], axis=1, inplace=True)
business_info['ZIP CODE'] = pd.to_numeric(business_info['ZIP CODE'], errors='coerce')
business_info.dropna(subset='ZIP CODE', inplace=True)
business_info = business_info[business_info['CITY'] == 'CHICAGO']
business_info = business_info[~business_info['APPLICATION TYPE'].isin(['C_LOC', 'C_SBA', 'C_EXPA', 'C_CAPA'])]

# keep only pertinent columns
business_info = business_info[['DOING BUSINESS AS NAME','LICENSE ID', 'ADDRESS', 'WARD', 'PRECINCT', 'POLICE DISTRICT', 'LICENSE CODE', 'LICENSE DESCRIPTION', 'LICENSE NUMBER','BUSINESS ACTIVITY ID', 'BUSINESS ACTIVITY', 
               'APPLICATION TYPE','APPLICATION REQUIREMENTS COMPLETE', 'CONDITIONAL APPROVAL', 'LICENSE TERM START DATE', 
               'LICENSE TERM EXPIRATION DATE', 'LICENSE APPROVED FOR ISSUANCE', 'DATE ISSUED']]

business_info.columns = ['aka_name', 'license_id', 'address', 'ward', 'precint',
                           'police_district', 'license_code',
                           'license_description', 'license_num', 'bus_activity_id',
                           'bus_activity', 'application_type',
                           'app_complete', 'conditional_approval',
                           'license_start_date', 'license_end_date',
                           'license_issued_date', 'date_issued']

# fixes most missing start date issues
business_info = business_info.assign(approx_start_date = lambda x: date_select(x))

# fixes about 2500 start date issues - date_issued is not as good of a metric - often off by 1 year
temp = business_info[business_info['approx_start_date'].isna()]  
temp['approx_start_date'] = temp['date_issued']
# remove temp records from business_info and add updated records
business_info.drop(labels=temp.index, axis=0, inplace=True)
business_info = pd.concat([business_info, temp], axis=0)


business_info['year'] = business_info['approx_start_date'].dt.strftime('%Y')  

# strip white space from object types
columns = business_info.select_dtypes(['object']).columns
business_info[columns] = business_info[columns].apply(lambda x: x.str.strip())

# create unique license num, year combo
business_info.drop_duplicates(subset=['aka_name', 'address', 'ward', 'precint',
       'police_district', 'license_code', 'license_description', 'license_num',
       'bus_activity_id', 'bus_activity','conditional_approval','year'], keep='last', inplace=True)

temp = business_info[business_info.duplicated(subset=['license_num','year'], keep=False).sort_values(ascending=True)].sort_values(by='aka_name', ascending=True)

# many 
idx = {}
description = {}
for index, row in temp.iterrows():
    if isinstance(row['bus_activity'], str):
        store0 = row['bus_activity_id']
        store1 = row['bus_activity']
        idx[row['license_num']] = store0
        description[row['license_num']] = store1

temp['bus_activity_id'] = temp['license_num'].map(idx)
temp['bus_activity'] = temp['license_num'].map(description)

business_info.drop(labels=temp.index, axis=0, inplace=True)
business_info = pd.concat([business_info, temp], axis=0)

business_info.drop_duplicates(subset=['aka_name', 'address', 'ward', 'precint',
       'police_district', 'license_code', 'license_description', 'license_num',
       'bus_activity_id', 'bus_activity','conditional_approval','year'], keep='last', inplace=True)

# remove all remaining missing date data - removes < 100 records
business_info.dropna(subset='approx_start_date', inplace=True)

# remove anything without an aka_name
business_info.dropna(subset=['aka_name'], axis=0, inplace=True)

business_info['year'] = business_info['year'].astype('int')

print(f'Number of records after cleaning:  {len(business_info)}')

business_info = business_info[['aka_name', 'license_id', 'address', 'ward', 'precint',
       'police_district', 'license_code', 'license_description', 'license_num',
       'bus_activity_id', 'bus_activity', 'application_type',
       'conditional_approval', 'approx_start_date', 'year']]

 # 48% of the business_activity and business_activity_id are NaN
business_info['bus_activity_id'].fillna('Unknown', inplace=True)
business_info['bus_activity'].fillna('Unknown', inplace=True)

# Calculate business age
min_max_year = business_info[['aka_name','address','year']].groupby(['aka_name','address']).agg({'year':['min','max']})
min_max_year.reset_index(inplace=True)
df = pd.DataFrame(min_max_year.to_records())
df.drop(labels=['index'], axis=1, inplace=True)
df.columns=['aka_name', 'address', 'year_min', 'year_max']

# Combine summarized results to with original dataframe
business_info = pd.merge(business_info, df, left_on=['aka_name', 'address'], right_on=['aka_name', 'address'], how='left')
business_info['bus_age'] = business_info['approx_start_date'].dt.year - business_info['year_min']
business_info.head()

business_info.head(2)

Original number of records: 1109521
Number of records after cleaning:  965046


,aka_name,license_id,address,ward,precint,police_district,license_code,license_description,license_num,bus_activity_id,bus_activity,application_type,conditional_approval,approx_start_date,year,year_min,year_max,bus_age
0,"AMERICAN ARBITRATION ASSOCIATION, INC.",2479692,150 N MICHIGAN AVE 30TH 3050,34.0,9.0,1.0,1010,Limited Business License,2354557.0,602,Administrative Commercial Office,RENEW,N,2016-09-16,2016,2016,2022,0
1,"FEDEX OFFICE AND PRINT SVCS, INC.",1940503,2301 S DR MARTIN LUTHER KING JR DR SOUTH BL,4.0,3.0,1.0,1010,Limited Business License,1847540.0,Unknown,Unknown,RENEW,N,2009-01-16,2009,2007,2023,2


In [9]:
business_info.dtypes

aka_name                        object
license_id                       int64
address                         object
ward                           float64
precint                        float64
police_district                float64
license_code                     int64
license_description             object
license_num                    float64
bus_activity_id                 object
bus_activity                    object
application_type                object
conditional_approval            object
approx_start_date       datetime64[ns]
year                             int32
year_min                         int32
year_max                         int32
bus_age                          int64
dtype: object

In [10]:
business_info.shape

(965046, 18)

## Merge Both Datasets Together

In [11]:
# check that there are no duplicates in the right merge table based on the keys - this will cause duplication
business_info_nodupes = business_info.drop_duplicates(subset=['license_num', 'year'], keep='first')
# business_info_nodupes.groupby(['license_num','year']).count()['aka_name'].sort_values(ascending=False)
# # shows that there are only single records for each license number and year combination.

# Merge shows it still has 150k rows but some are NaN values
restaurant_df = pd.merge(restaurant_df, business_info_nodupes, left_on=['license_num','year'], right_on=['license_num','year'], how='left')
# restaurant_df.shape

# the first merge was successful for 16,000 rows but 79,000 rows were still mostly Nan
temp = restaurant_df[restaurant_df.isna().sum(axis=1) > 8][['inspect_id', 'aka_name_x', 'license_num', 'facility_type', 'risk',
       'address_x', 'city', 'state', 'zipcode', 'inspect_date', 'inspect_type',
       'results', 'violations', 'lat', 'lon', 'violations_list',
       'violation_count', 'violation_number', 'vl_must_comply_list',
       'vl_must_comply_count', 'vl_instructed_comply_list', 'vl_instructed_comply_count',
       'vl_not_cited_list', 'vl_citation_count', 'year', 'month']]

temp.reset_index(inplace=True)

# Strategey to combine more records is to reduce the specificity of the merge 
# First isolate the NaN values (> 8 NaN values in a row) 
# Merge to this limted dataframe, first on license number alone, then drop those indexes from the original dataframe and add the updated data to the original
# Next isolate the NaN values, and merge on name and address which will be less accurate then drop index from original dateaframe and add updated data to the original
# Important note:  make sure that right table of the merge has not duplcates for the merging keys - this prevents duplication of records in the original dataframe

# # this verifies that the right table does not have duplicates
business_info_nodupes2 = business_info_nodupes.drop_duplicates(subset=['license_num'], keep='first')
# business_info_nodupes2.groupby(['license_num']).count()['aka_name'].sort_values(ascending=False)

# the second merge was successful for 77,000 rows but 1,500 rows were still mostly Nan
temp2 = pd.merge(temp, business_info_nodupes2, left_on=['license_num'], right_on=['license_num'], how='left')
temp2.set_index('index', inplace=True)

temp2.drop(labels='year_y', axis=1, inplace=True)
temp2.rename(columns={'year_x':'year', 'aka_name':'aka_name_y', 'address':'address_y'}, inplace=True)

# remove temp records from restaurant_df and add updated records
restaurant_df.drop(labels=temp2.index, axis=0, inplace=True)
restaurant_df = pd.concat([restaurant_df, temp2], axis=0)

temp3 = temp2[temp2.isna().sum(axis=1) > 8]
# remove the final mismatches since there should be a match on license number
restaurant_df.drop(labels=temp3.index, axis=0, inplace=True)

# drop non-name records ~700 records
restaurant_df.dropna(subset='aka_name_x', inplace=True, axis=0)

# create a new license number for every name/address combination
temp = restaurant_df.groupby(['aka_name_x', 'address_x']).min()[['license_num']]
name_address = temp.index
new_license_number = {(i,j): k for k,(i,j) in enumerate(name_address)}

# add license alias to dataframe
restaurant_df['license_alias'] = restaurant_df.apply(lambda x: new_license_number[x.aka_name_x,x.address_x], axis=1)

# add number of chains that exist
temp = restaurant_df.groupby('aka_name_x').count()
names = temp.index

number_of_chains ={}
for name in names:
    num = restaurant_df[restaurant_df['aka_name_x']== name]['license_alias'].nunique()  
    number_of_chains[name] = num
    
restaurant_df['number_of_chains'] = restaurant_df.apply(lambda x: number_of_chains[x.aka_name_x], axis=1)

restaurant_df['week'] = restaurant_df['inspect_date'].dt.week  

restaurant_df['inspections_per_year'] = restaurant_df.groupby('license_alias').transform('count')['inspect_id']

def restaurant_type(num_chains):
    if num_chains == 1:  
        val = 'Single Site'
    elif num_chains <= 5: 
        val = '2-5 Sites'
    elif num_chains <=10:
        val = '6-10 Sites'
    elif num_chains <= 25:
        val = '11-25 Sites'
    else:
        val = '26+ Sites'
    return val
        
restaurant_df['type_of_restaurant'] = restaurant_df['number_of_chains'].apply(lambda x: restaurant_type(x))

# not a fast calculation
Audio(sound_file, autoplay=True) 

restaurant_df.head()

,inspect_id,aka_name_x,license_num,facility_type,risk,address_x,city,state,zipcode,inspect_date,inspect_type,results,violations,lat,lon,violations_list,violation_count,violation_number,vl_must_comply_list,vl_must_comply_count,vl_instructed_comply_list,vl_instructed_comply_count,vl_not_cited_list,vl_citation_count,year,month,aka_name_y,license_id,address_y,ward,precint,police_district,license_code,license_description,bus_activity_id,bus_activity,application_type,conditional_approval,approx_start_date,year_min,year_max,bus_age,license_alias,number_of_chains,week,inspections_per_year,type_of_restaurant
3,2579789,SENOR PAN CAFE,2659113.0,Restaurant,Risk 1 (High),4612 W FULLERTON AVE,CHICAGO,IL,60639.0,2023-08-09,Canvass Re-Inspection,Pass,-99. No violations reported,41.924402,-87.742318,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8,SENOR PAN CAFE,2899694.0,4612 W FULLERTON AVE 1,31.0,29.0,25.0,1006.0,Retail Food Establishment,735,Preparation of Food and Dining on Premise With...,RENEW,N,2023-05-16,2019.0,2023.0,4.0,14065,5,32,12,2-5 Sites
4,2579779,TAQUERIA CD HIDALGO,2078867.0,Restaurant,Risk 1 (High),7104 N CLARK ST,CHICAGO,IL,60626.0,2023-08-08,Canvass,Pass,-99. No violations reported,42.011581,-87.674614,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8,TAQUERIA CD HIDALGO,2884398.0,7104 N CLARK ST 1ST,49.0,3.0,24.0,1006.0,Retail Food Establishment,775,Retail Sales of Perishable Foods,RENEW,N,2023-02-16,2011.0,2023.0,12.0,15894,1,32,24,Single Site
7,2579723,BAR COCINA,2334704.0,Restaurant,Risk 1 (High),2901 N SHEFFIELD AVE,CHICAGO,IL,60657.0,2023-08-08,Canvass Re-Inspection,Pass,-99. No violations reported,41.934501,-87.653775,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8,BAR COCINA,2898722.0,2901 N SHEFFIELD AVE 1,44.0,29.0,19.0,1006.0,Retail Food Establishment,735,Preparation of Food and Dining on Premise With...,RENEW,N,2023-05-16,2015.0,2023.0,8.0,1121,1,32,17,Single Site
8,2579643,Itoko / GG'S Chicken Shop / Little Goat Diner,2872999.0,Restaurant,Risk 1 (High),3323-3325 N SOUTHPORT AVE,CHICAGO,IL,60657.0,2023-08-07,Complaint Re-Inspection,Pass,-99. No violations reported,41.942334,-87.663739,[-99. No violations reported],1,[99],[False],0,[False],0,[False],1,2023,8,Itoko / GG's Chicken Shop / Little Goat Diner,2872999.0,3323-3325 N SOUTHPORT AVE LL-2,44.0,NaN,NaN,1006.0,Retail Food Establishment,735,Preparation of Food and Dining on Premise With...,ISSUE,Y,2023-02-21,2023.0,2023.0,0.0,7640,1,32,6,Single Site
10,2579561,JIN JU,27137.0,Restaurant,Risk 1 (High),5203 N CLARK ST,CHICAGO,IL,60640.0,2023-08-04,Canvass,Pass,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.976301,-87.668276,[10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPL...,4,"[10, 47, 55, 58]","[False, False, False, False]",0,"[False, False, False, False]",0,"[False, False, False, False]",4,2023,8,JIN JU,2907977.0,5203 N CLARK ST 1ST,48.0,10.0,20.0,1006.0,Retail Food Establishment,775,Retail Sales of Perishable Foods,RENEW,N,2023-07-16,2001.0,2023.0,22.0,8059,1,31,19,Single Site


In [12]:
restaurant_df.shape

(147908, 47)

In [13]:
restaurant_df.dtypes

inspect_id                             int64
aka_name_x                            object
license_num                          float64
facility_type                         object
risk                                  object
address_x                             object
city                                  object
state                                 object
zipcode                              float64
inspect_date                  datetime64[ns]
inspect_type                          object
results                               object
violations                            object
lat                                  float64
lon                                  float64
violations_list                       object
violation_count                        int64
violation_number                      object
vl_must_comply_list                   object
vl_must_comply_count                   int64
vl_instructed_comply_list             object
vl_instructed_comply_count             int64
vl_not_cit

In [14]:
# Note:  approximate_start_date should probably not be used; there is potential that the other categoricals could be wrong

In [15]:
# # export to csv
restaurant_df.to_csv('./data/manipulated/combined_data.csv')